# face detect

In [1]:
# Python 2/3 compatibility
from __future__ import print_function

import numpy as np
import cv2 as cv
from PIL import Image
# local modules
from video import create_capture
from common import clock, draw_str


from keras.models import model_from_json
from keras import layers
from keras import models
from keras.preprocessing import image
from tensorflow.python.framework import ops
ops.reset_default_graph()

import os

Using TensorFlow backend.


In [2]:
#detect
def detect(img, cascade):
    rects = cascade.detectMultiScale(img, scaleFactor=1.3, minNeighbors=4, minSize=(30, 30),
                                     flags=cv.CASCADE_SCALE_IMAGE)
    if len(rects) == 0:
        return []
    rects[:,2:] += rects[:,:2]
    #print(rects[0][2]-rects[0][0], rects[0][3]-rects[0][1])
    return rects

#when detect, this reacts
def draw_rects(img, rects, color):
    for x1, y1, x2, y2 in rects:
        cv.rectangle(img, (x1, y1), (x2, y2), color, 2)
        
def onlyface(img, rects):
    for x1, y1, x2, y2 in rects:
        #print(x1, y1, x2, y2)
        x1 = max(0, x1 - 80)
        x2 = min(480, x2+80)
        y1 = max(0, y1-80)
        y2 = min(640, y2+80)
        return img[y1:y2,x1:x2]
    return None

In [3]:
import easydict
import argparse

In [4]:
#####################################################################
parser = argparse.ArgumentParser()
parser.add_argument('--cascade', type=str, default='./haarcascade_frontalface_alt.xml',
                    help='cascade file')
parser.add_argument('--nested-cascade', type=str,
                    default='./haarcascades/haarcascade_eye.xml',
                    help='nested-cascade file')
args = parser.parse_args()
#####################################################################

usage: ipykernel_launcher.py [-h] [--cascade CASCADE]
                             [--nested-cascade NESTED_CASCADE]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\LGJ\AppData\Roaming\jupyter\runtime\kernel-73f0a314-c6ea-46c2-8dc3-9f81f3bbbe1c.json


SystemExit: 2

C:\Users\LGJ\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3333: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [11]:
args = easydict.EasyDict({ "cascade" : "./haarcascade_frontalface_alt.xml",
                          "nested_cascade" : "./haarcascade_eye.xml"
                         })

In [12]:

print('----- info -----')
print('[i] The config file: ', args.cascade)
print('[i] The weights of model file: ', args.nested_cascade)
print('###########################################################\n')

----- info -----
[i] The config file:  ./haarcascade_frontalface_alt.xml
[i] The weights of model file:  ./haarcascade_eye.xml
###########################################################



In [13]:
dirname = 'data'
output = 'output'

In [14]:
filenames = os.listdir(dirname)
absFileNmaes = []
absOutoutNames = []
for i in filenames:
    absFileNmaes.append(os.path.join(dirname, i))
    absOutoutNames.append(os.path.join(output, i))

In [15]:
def main():
    import sys, getopt

    cascade_fn = args.cascade
    nested_fn  = args.nested_cascade

    cascade = cv.CascadeClassifier(cv.samples.findFile(cascade_fn))
    nested = cv.CascadeClassifier(cv.samples.findFile(nested_fn))

    
    for k in range(len(absFileNmaes)):
        i = absFileNmaes[k]
        j = absOutoutNames[k]
        cap = cv.VideoCapture(i)
        ret, img = cap.read()

        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        gray = cv.equalizeHist(gray)


        rects = detect(gray, cascade)
        vis = img.copy()
        draw_rects(vis, rects, (0, 255, 0))



        if not nested.empty():
            for x1, y1, x2, y2 in rects:
                roi = gray[y1:y2, x1:x2]
                vis_roi = vis[y1:y2, x1:x2]
                subrects = detect(roi.copy(), nested) #detect
                draw_rects(vis_roi, subrects, (255, 0, 0))
                cv.imwrite(j, vis_roi.astype(np.uint8))


    print('Done')


if __name__ == '__main__':
    print(__doc__)
    main()
    cv.destroyAllWindows()


Automatically created module for IPython interactive environment
Done
